In [1]:
from dipy.viz import window, actor
from dipy.data import fetch_bundles_2_subjects, read_bundles_2_subjects
from dipy.tracking.streamline import transform_streamlines

import deepdish as dd
import numpy as np

import json

In [1]:
import numpy as np
from dipy.viz import window, actor
from dipy.data import fetch_bundles_2_subjects, read_bundles_2_subjects
from dipy.tracking.streamline import transform_streamlines

fetch_bundles_2_subjects()
dix = read_bundles_2_subjects(subj_id='subj_1', metrics=['fa'],
                              bundles=['cg.left', 'cst.right'])

Data size is approximately 234MB
Dataset is already in place. If you want to fetch it again please first remove the folder /Users/richford/.dipy/exp_bundles_and_maps 


In [2]:
fa = dix['fa']

In [3]:
affine = dix['affine']

In [4]:
bundle = dix['cg.left']

In [5]:
bundle_native = transform_streamlines(bundle, np.linalg.inv(affine))

In [6]:
renderer = window.Renderer()

stream_actor = actor.line(bundle_native)

renderer.set_camera(position=(-176.42, 118.52, 128.20),
                    focal_point=(113.30, 128.31, 76.56),
                    view_up=(0.18, 0.00, 0.98))

renderer.add(stream_actor)

In [70]:
cmap.min()

0.004874

In [72]:
fa.shape

(256, 256, 150)

In [77]:
bundle_native

(70, 3)

In [71]:
renderer.clear()

cmap = actor.create_colormap(np.ravel(fa), name="viridis")

stream_actor2 = actor.line(bundle_native, cmap, linewidth=0.1)
bar = actor.scalar_bar()

renderer.add(stream_actor2)
renderer.add(bar)

renderer.set_camera(position=(-176.42, 118.52, 128.20),
                    focal_point=(113.30, 128.31, 76.56),
                    view_up=(0.18, 0.00, 0.98))

axes = actor.axes(scale=(20, 20, 20))
axes.SetPosition((
    np.mean(stream_actor2.GetXRange()),
    np.max(stream_actor2.GetYRange()) - 20,
    np.min(stream_actor2.GetZRange()) - 20,
))
renderer.add(axes)

window.show(renderer, size=(600, 600), reset_camera=False)

In [17]:
stream_actor2.GetPosition()

(0.0, 0.0, 0.0)

In [30]:
coord = bar.GetActualPositionCoordinate()
coord2 = bar.GetActualPosition2Coordinate()

# Load streamlines (common to both plots)

In [2]:
with open("../data/raw/core_streamlines.json") as fp:
    cores = json.load(fp)

# Load and plot $\widehat{\beta}$ for ALS classification

In [3]:
als_betas_by_group = dd.io.load("als_classify_betas.h5")

In [4]:
metric = "fa"

renderer = window.Renderer()
renderer.background((1, 1, 1))
colors = np.concatenate([als_betas_by_group[bn][metric]
                         for bn in als_betas_by_group.keys()])
bundle_names = als_betas_by_group.keys()
bundles = [np.array(cores[bn]["coreFiber"]) for bn in bundle_names]
stream_actor = actor.line(
    lines=bundles,
    colors=actor.create_colormap(colors, name='PuOr', auto=True),
    linewidth=20,
    spline_subdiv=300,
    fake_tube=True,
)

renderer.add(stream_actor)
window.show(renderer, size=(900, 900), reset_camera=True)

# Load and plot $\widehat{\beta}$ for age regression

In [5]:
age_betas_by_group = dd.io.load("age_regression_betas.h5")

In [6]:
renderer = window.Renderer()

metric = "fa"
colors = np.concatenate([age_betas_by_group[bn][metric] for bn in age_betas_by_group.keys()])
bundle_names = age_betas_by_group.keys()
bundles = [np.array(cores[bn]["coreFiber"]) for bn in bundle_names]
stream_actor = actor.line(
    lines=bundles,
    colors=actor.create_colormap(colors, name='coolwarm'),
    linewidth=20,
    spline_subdiv=300,
    fake_tube=True,
)

renderer.add(stream_actor)
window.show(renderer, size=(900, 900), reset_camera=True)